- Gets a basic list of Entities (by Model or by Business Area) and their relations
    
- Builds JSON to feed a Jinja template for either Basic ER or Chen's ER
    
- Does not do types/ group entities
    
- declarations
    
- get table of conceptual entities in target model, along with the groups they are in
    
- put them in a json dict
    
- get table of links between conceptual entities
    
- put them in a json dict
    
- spin the json dict into the jinja template for yed  
      
    Currently, this does output a diagram, that does work, but it's pretty flaky :(  
    Debugging the diagram/ graphml is going to be a far bigger job.

In [1]:
import json
import os
import sqlite3

from pathlib import Path
from jinja2 import Template, Environment
from datetime import datetime

conn = sqlite3.connect('../full_metadata.db')
cur = conn.cursor()

template_path = "../export_templates/yed_live/"
template_filename = "conceptual_er.tem"
template = Path(os.path.join(template_path, template_filename)).read_text()

time_string = datetime.now().strftime('%Y%m%d%H%M%S')
output_path = "../export_output_files/yed_live/"
output_filename = "conceptual_er_" + time_string + ".graphml"

data = {}


In [2]:
# Script variables
target_model_name = ''  # ER model to output, leave blank for 'all'
target_business_name = 'Vivid' # Business to target, leave blank for 'all'
target_business_area_name = 'Repairs' # Business area to target, leave blank for 'all'

output_template_type = 'Basic' # One of 'Basic', 'Chens'

In [3]:
def fetch_data_as_dict(cursor, query, params):
  try:
    cursor.execute(query, params)
    columns = [desc[0] for desc in cursor.description]
    results = cursor.fetchall()
    data_dict_list = [dict(zip(columns, row)) for row in results]
    return data_dict_list
  except sqlite3.Error as e:
    print(f"Error: {e}")
    return []

In [4]:
# Get nodes
# Which are a bit more complicated than edges

where_clause = '';
params = []

if (target_model_name != ''):
  where_clause = where_clause + ' AND ModelKeyPhrase = ? '
  params = params + [target_model_name]

if (target_business_name != ''):
  where_clause = where_clause + ' AND BusinessName = ? '
  params = params + [target_business_name]

if (target_business_area_name != ''):
  where_clause = where_clause + ' AND BusinessAreaName = ? '
  params = params + [target_business_area_name]

params = params + params

sql_query = """
SELECT DISTINCT
    ConceptualEntityHashKey
  , CE_DisplayName		
FROM out_ConceptualEntityRelationsForYed
WHERE
  1=1
  """ + where_clause + """

UNION

SELECT DISTINCT
    ConceptualEntity_RelatedHashKey
  , CER_DisplayName		
FROM out_ConceptualEntityRelationsForYed
WHERE
  1=1
  """ + where_clause + """
""";

nodes_dict_list = fetch_data_as_dict(cur, sql_query, params)

#print(nodes_dict_list)

In [5]:
# Get edges

where_clause = '';
params = []

if (target_model_name != ''):
  where_clause = where_clause + ' AND ModelKeyPhrase = ? '
  params = params + [target_model_name]

if (target_business_name != ''):
  where_clause = where_clause + ' AND BusinessName = ? '
  params = params + [target_business_name]

if (target_business_area_name != ''):
  where_clause = where_clause + ' AND BusinessAreaName = ? '
  params = params + [target_business_area_name]


sql_query = """
SELECT DISTINCT
    ConceptualEntityHashKey
  , CE_DisplayName
  , ForwardRelationPhrase
  , ConceptualEntity_RelatedHashKey
  , CER_DisplayName
FROM out_ConceptualEntityRelationsForYed
WHERE
  1=1
  """ + where_clause + """
""";

edges_dict_list = fetch_data_as_dict(cur, sql_query, params)

#print(edges_dict_list)

In [6]:

data['nodes'] = nodes_dict_list
data['edges'] = edges_dict_list

In [7]:
# drop the JSON into the Template

j2_template = Template(template, autoescape=True)

rendered_template_string = j2_template.render(data)
with open(os.path.join(output_path, output_filename), "w") as text_file:
  text_file.write(rendered_template_string)